#### Devido a organização das pastas, uso as bibliotecas "os" e "sys" para definidir o local como sendo na pasta anterior, a "Controle_jubilee"

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

#### Em seguida, faço a importação os objetos e funções dos scripts Python. Recomendo que acesse esses arquivo .py para conferir melhor as 

In [2]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle

In [3]:
jubilee = JubileeMotionController()

In [7]:
jubilee.reset()

Reconnecting...


In [4]:
jubilee.home_all(mesh_mode_z=False)

In [5]:
jubilee.move_xyz_absolute(z=120)

In [7]:
jubilee.controlar_jubilee()

X:  160.00 | Y:  200.00 | Z:   70.00 || Passo: 10  mm
 Interrompido


In [ ]:
jubilee.axis_limits

[(0, 200.0), (0, 200.0), (0, 200.0), (0, 200.0)]

In [6]:
import lgpio

In [7]:
class Gripper:
    def __init__(self,machine,servo_pin=13):

        self.handle = lgpio.gpiochip_open(0)
        self.servo_pin = servo_pin
        self.machine = machine
        self.move_velocity = 10000

        self.machine.gcode("M208 Z100:300")
        self.machine.move_xyz_absolute(z=100)
        

    def install(self):
        """
        Instala a ferramenta da câmera na máquina.

        Este método move o cabeçote para as coordenadas específicas
        necessárias para acoplar a câmera ao sistema Jubilee. As
        Coodernadas foram descobertas empíricamente e podem ser 
        alteradas se necessário.
        """
        self.machine.protect_tools(on=False)
        
        self.machine.move_xyz_absolute(y=220, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=40, velocity=self.move_velocity)
        self.machine.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=7, velocity=self.move_velocity)
        self.machine.gcode("G0 U0")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=70, y=120, velocity=self.move_velocity)

        if self.machine.mode_protect_tools:
            self.machine.protect_tools(on=False)

    def uninstall(self):
        """
        Remove a ferramenta da câmera da máquina.

        Este método move o cabeçote para as coordenadas específicas
        necessárias para desacoplar a câmera do sistema Jubilee.
        """
        self.machine.protect_tools(on=False)

        self.machine.move_xyz_absolute(y=90, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=40, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(y=7, velocity=self.move_velocity)
        self.machine.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=70, y=120, velocity=self.move_velocity)
        self.machine.gcode("G0 U0")

        if self.machine.mode_protect_tools:
            self.machine.protect_tools(on=False)



    def set_angle(self,angle):
        # Converte ângulo para duty cycle (0°=2.5%, 180°=12.5%)
        duty = 2.5 + (angle / 180.0) * 10.0
        lgpio.tx_pwm(self.handle, self.servo_pin, 50, duty)

    def end(self):
        lgpio.gpiochip_close(self.handle)
    
    def begin(self):
        self.handle = lgpio.gpiochip_open(0)

    
    def open(self):
        self.begin()
        lgpio.gpio_claim_output(self.handle, self.servo_pin)
        self.set_angle(30)
        time.sleep(0.5)
        self.end()
    
    def open_angles(self,angle):
        self.begin()
        lgpio.gpio_claim_output(self.handle, self.servo_pin)
        self.set_angle(angle)
        time.sleep(0.5)
        self.end()

    def close(self):
        self.begin()
        lgpio.gpio_claim_output(self.handle, self.servo_pin)
        self.set_angle(0)
        time.sleep(0.5)
        self.end()

In [8]:
gri = Gripper(jubilee)
camera = camera_tool(jubilee)

In [12]:
for _ in range(0,3):
    gri.install()
    time.sleep(1)
    gri.uninstall()

In [10]:
jubilee.protect_tools()

TypeError: JubileeMotionController.protect_tools() missing 1 required positional argument: 'on'

In [11]:
for _ in range(0,10):
    camera.install()
    time.sleep(1)
    camera.uninstall()
    time.sleep(1)
    gri.install()
    time.sleep(1)
    gri.uninstall()

KeyboardInterrupt: 

In [51]:
jubilee.move_xyz_absolute(z=220)

In [35]:
gri.install()

In [25]:
for _ in range(5):
    time.sleep(2)
    gri.close()
    jubilee.move_xyz_absolute(z=150,velocity=15000)
    jubilee.move_xyz_absolute(x=100,y=240,velocity=15000)
    jubilee.move_xyz_absolute(z=100,velocity=15000)
    time.sleep(5)
    gri.open()
    jubilee.move_xyz_absolute(z=150,velocity=15000)
    jubilee.move_xyz_absolute(x=40,y=300,velocity=15000)
    jubilee.move_xyz_absolute(x=100,y=240,velocity=15000)
    jubilee.move_xyz_absolute(z=100,velocity=15000)
    time.sleep(5)
    gri.close()
    jubilee.move_xyz_absolute(z=150,velocity=15000)


    jubilee.move_xyz_absolute(x=250,y=100,velocity=15000)
    jubilee.move_xyz_absolute(z=120,velocity=15000)
    time.sleep(5)
    gri.open()
    jubilee.move_xyz_absolute(z=100,velocity=15000)


In [51]:
jubilee.move_xyz_absolute(x=100,y=180,z=110)

In [52]:
gri.open_angles(90)

In [47]:
jubilee.keyboard_controll()

X:  120.00 | Y:  270.00 | Z:  126.00 || Passo: 10  mm
 Interrompido


In [64]:
jubilee.move_xyz_absolute(z=250,velocity=15000)

In [62]:
jubilee.move_xyz_absolute(z=160,velocity=1000)

In [53]:
gri.uninstall()